# Tutorial by Marc LABADIE for software paper rebuild echap loop with adel protocol after conversion in python3

## 1. Adel:
Allows to simulate the 3D architectural development of the shoot of gramineaous plant. 
* Adel-Maize (Fournier & Andrieu, 1998)
* Adel-Wheat (Fournier et al. 2003) models

### 1.1 Create a wheat architectural model with geometry at a given development stage and visualize wheat

In [ ]:
from alinea.adel.astk_interface import AdelWheat
import openalea.plantgl.all as pgl
from oawidgets.plantgl import PlantGL

%gui qt
#%pdb

In [ ]:
adel = AdelWheat(nplants=2) # load architectural rule and parameters of wheat
g = adel.setup_canopy(age=1500) # Select the age of wheat, age represent degree days
adel.plot(g)


### 1.2 Create an wheat architectural model at given developmental stage from data (Echap data)

In [ ]:
from alinea.echap.architectural_reconstructions import EchapReconstructions

In [ ]:
echap = EchapReconstructions()

# load reconstruction of Tremie12 variaty at 1500 degree days
wheat_Mercia = echap.get_reconstruction(name="Mercia", nplants=30,seed=0)
g_Mercia= wheat_Mercia.setup_canopy(age=1501)
PlantGL(wheat_Mercia.plot(g_Mercia), group_by_color=False)

In [ ]:
# load reconstruction of Tremie12 variaty at 1500 degree days
wheat_Tremie12 = echap.get_reconstruction(name="Tremie12", nplants=30,seed=0)
g_Tremie12= wheat_Tremie12.setup_canopy(age=1569)
PlantGL(wheat_Tremie12.plot(g_Tremie12), group_by_color=False)

### 1.3 It possible to simulate grow canopy using different age of the canopy or directely using weather data (temperature)

#### 1.3.1 Using canopy age:

* canopy_age initiat at 100 degree days
* grow_cycle: number of grow cycle
* grow_time: interval of degree days between each cycle (100)

In [ ]:
canopy_age= 100 # age of canopy at start
grow_cycles =20 # nb of growth cycle
grow_time = 100 #degree day, time interval between each cycle

wheat = echap.get_reconstruction(name="Tremie12", nplants=2,seed=0) # init a wheat mtg

for cycle in range(grow_cycles):
    canopy_age = canopy_age
    # update canopy
    new_g = wheat.setup_canopy(age=canopy_age)
    wheat.plot(new_g)
    canopy_age = canopy_age + grow_time

#### 1.3.2 With weather data issue from echap project

**selection of Weather parameter**

Based on weather data:
* weather_time_index: selection of the period where is applied simulation
* every_dd: among the period selected filter data by step of the thermal time here each 15 degree days 

In [ ]:
from alinea.echap.weather_data import get_weather

from alinea.alep.mini_models import linear_degree_days
from alinea.alep.protocol import update
from alinea.alep.growth_control import NoPriorityGrowthControl

from alinea.astk.TimeControl import DegreeDayModel, thermal_time_filter
from alinea.astk.TimeControl import IterWithDelays , time_control

import pandas

In [ ]:
# import weather data
weather = get_weather(variety="Mercia") # get weather data from echap for Mercia variety

#check weather data and add degree day variable using linear degree day model
weather.check(["degree_days",'temperature_air', 'PPFD', 'relative_humidity', 'wind_speed', 'rain', 'global_radiation', 'vapor_pressure'],models={'degree_days':linear_degree_days})

# weather index selection
weather_time_index = pandas.date_range(start ="2010-11-02", end='2011-05-09', freq='H')  # Range date index. among weather data select weather period for simulation

# filter degree days
TTmodel = DegreeDayModel(Tbase = 0) # Thermal times model
every_dd = thermal_time_filter(time_sequence=weather_time_index, weather=weather, model=TTmodel, delay = 15) # allows to filter thermal time data here each 15 degree day
canopy_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_dd, data=weather.data)) # create data from weatherdata according to filter

# growth control
growth_controler = NoPriorityGrowthControl()

In [ ]:
wheat = echap.get_reconstruction(name="Mercia", nplants=2,seed=0)
g = wheat.setup_canopy(age=500)

for i,controls in enumerate(canopy_timing):
    canopy_iter = controls

    if canopy_iter.eval:
        print('--', canopy_iter.value.index[-1], '--')
        print('-- update microclimate / canopy --')
        g = wheat.grow(g, canopy_iter.value)
        _=update(g, canopy_iter.dt, growth_controler)
        wheat.plot(g)


## 2. ALEP: Interaction Plant/Pathogene

In [1]:
# import for Architectural construction
from alinea.echap.architectural_reconstructions import EchapReconstructions

# import weather
from alinea.echap.weather_data import get_weather
from alinea.alep.mini_models import linear_degree_days, leaf_wetness_rapilly
from alinea.astk.TimeControl import DegreeDayModel, thermal_time_filter, time_filter,rain_filter
from alinea.astk.TimeControl import IterWithDelays , time_control

# import for canopy
from alinea.astk.plant_interface import grow_canopy
from alinea.alep.architecture import set_properties, update_healthy_area

# import for disease
from alinea.alep.septo3d_v2 import plugin_septoria
from alinea.alep.inoculation import RandomInoculation
from alinea.alep.dispersal_emission import SeptoriaRainEmission
from alinea.alep.dispersal_transport import SeptoriaRainDispersal
from alinea.alep.infection_control import BiotrophDUProbaModel
from alinea.alep.growth_control import NoPriorityGrowthControl
from alinea.alep.architecture import get_leaves
from alinea.alep.protocol import external_contamination, disperse, infect,update
from alinea.alep.disease_operation import DU_Generator, generate_stock_du
from alinea.echap.contamination import SimpleSoilInoculum, SimpleContamination
from alinea.alep.disease_outputs import plot_severity_by_leaf


from oawidgets.plantgl import PlantGL

import pandas
import random as rd
%gui qt

#### Model seul

In [2]:
echap = EchapReconstructions()
wheat = echap.get_reconstruction(name='Mercia',nplants=2)
g=wheat.setup_canopy(age=500)
wheat.plot(g)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to 

In [3]:
# import weather data
weather = get_weather(variety="Mercia") # get weather data from echap for Mercia variety

#check weather data and add degree day variable using linear degree day model
weather.check(['temperature_air', 'PPFD', 'relative_humidity', 'wind_speed', 'rain', 'global_radiation', 'vapor_pressure',"degree_days","wetness"],models={'degree_days':linear_degree_days,'wetness':lambda data: leaf_wetness_rapilly(data.rain, data.relative_humidity, data.PPFD)})

# weather index selection
weather_time_index = pandas.date_range(start ="2010-11-02", periods=500, freq='H')  # Range date index. among weather data select weather period for simulation
every_h = time_filter(time_sequence=weather_time_index, delay=1)
every_24h = time_filter(time_sequence=weather_time_index, delay=24)
every_rain= rain_filter(time_sequence=weather_time_index,weather= weather,rain_min=0.1)

weather_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_h, data=weather.data))
wheat_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_24h, data=weather.data))
septo_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_h, data=weather.data))
rain_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter= every_rain, data=weather.data))

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1124: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  return self._get_label(key, axis=axis)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


In [4]:
septoria = plugin_septoria()
inoculator = RandomInoculation() # Class to distributed randomly dispersal units
infection_controler = BiotrophDUProbaModel() #class for checking position of dispersal units on a MTG
emitter = SeptoriaRainEmission(domain_area=wheat.domain_area) #class for a model of emission of dispersal units by rain.mission is the first step of dispersal, before transport
transporter = SeptoriaRainDispersal() # model of dispersal by rain
growth_controler = NoPriorityGrowthControl()

In [42]:
import numpy as np
import random as rd
from alinea.alep.disease_outputs import AdelSeptoRecorder
g=g
group_dus=False
nb_steps=100

# Initiate wheat MTG with one leaf and good conditions for disease
set_properties(g, label='LeafElement', wetness=True, 
                temperature_sequence=weather.data.temperature_air[0], 
                degree_days=weather.data.degree_days[0],
                rain_intensity=0., 
                global_efficacy={'eradicant':0, 'protectant':0})

# Initiate disease
nb_dus=10
leaf = get_leaves(g)
leaf = g.node(12) # source leaf infection

dispersal_units = [septoria.dispersal_unit() for i in range(nb_dus)]

for du in dispersal_units:
    du.set_position([rd.random() * leaf.length, 0])

if group_dus:
    pos = [du.position[0] for du in dispersal_units]
    dispersal_units[0].fungus.group_dus = True
    dispersal_units[0].set_position(pos)
    dispersal_units = [dispersal_units[0]]

leaf.dispersal_units = dispersal_units

# Initiate output recorder
recorder = AdelSeptoRecorder(group_dus=False, increment=100)
recorder

# Simulation loop
for i in range(nb_steps):
    # Update wheat healthy area
    update_healthy_area(g, label = 'LeafElement')
    
    # Update dispersal units and lesions
    
    infect(g, 1, infection_controler, label='LeafElement')
    update(g, 1, growth_controler,  label='LeafElement')
    
    
    # Get outputs
    recorder.record(g, date=i+1)

In [47]:
set(recorder.data.)

{0.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan}

In [ ]:
nb_dus=20
for i, controls in enumerate(zip(weather_timing,wheat_timing,septo_timing, rain_timing)): 
    weather_eval, wheat_eval,septo_eval,rain_eval = controls

    # Get weather for date and add it as properties on leaves
    if weather_eval:
        print("microclimate")
        set_properties(g,label = 'LeafElement',
                       temp = weather_eval.value.temperature_air[0],
                       wetness = weather_eval.value.wetness[0],
                       relative_humidity = weather_eval.value.relative_humidity[0],
                       wind_speed = weather_eval.value.wind_speed[0])
    if rain_eval:
        set_properties(g,label = 'LeafElement',
                       rain_intensity = rain_eval.value.rain.mean(),
                       rain_duration = len(rain_eval.value.rain) if rain_eval.value.rain.sum() > 0 else 0.)
    
        # Grow wheat canopy
    if wheat_eval:
        print("grow canopy")
        g,_ = grow_canopy(g, wheat, wheat_eval.value)
        # Note : The position of senescence goes back to its initial value after
        # a while for undetermined reason
        # --> temporary hack for keeping senescence position low when it is over
        positions = g.property('position_senescence')
        greens = g.property('is_green')
        areas = g.property('area')
        senesced_areas = g.property('senesced_area')
        leaves = get_leaves(g, label = 'LeafElement')
        leaf=g.node(leaves[0])
       # vids = [leaf for leaf in leaves if leaf in g.property('geometry')]
       # positions.update({vid:(0 if (positions[vid]==1 and not greens[vid]) or
       #                             (positions[vid]>0 and round(areas[vid],5)==round(senesced_areas[vid],5))
       #                             else positions[vid]) for vid in vids})
        dispersal_units = [septoria.dispersal_unit() for i in range(nb_dus)]
        for du in dispersal_units:
            du.set_position([rd.random() * leaf.length, 0])
            
        _ = update(g, wheat_eval.dt, growth_controler)
        #wheat.plot(g)
    
        # Develop disease
    if septo_eval:
        # Update g for the disease
        #sen_model.find_senescent_lesions(g, label = 'LeafElement')
        update_healthy_area(g, label = 'LeafElement')
        
        # Update dispersal units and lesions
        infect(g, septo_eval.dt, infection_controler, label='LeafElement')
        update(g, septo_eval.dt, growth_controler, label='LeafElement')
        
    
    # Disperse and wash
    if rain_eval:
        if rain_eval.value.rain.mean()>0:
            g = disperse(g, emitter, transporter, "septoria", label='LeafElement')
        plot_severity_by_leaf(g, senescence=False, transparency=0.9)

In [ ]:
seporia_sol=0.01 # level of septoria on the sol
soil_inoculation = SimpleSoilInoculum(DU_generator=DU_Generator(disease=septoria), sporulating_fraction =seporia_sol, domain_area = wheat.domain_area, convUnit=wheat.convUnit)
contaminator = SimpleContamination(domain = wheat.domain, domain_area =wheat.domain_area, convUnit=wheat.convUnit)
g = external_contamination(g=g, contamination_source=soil_inoculation, contamination_model=contaminator, weather_data= weather.data, label='LeafElement')

In [ ]:
g.property('dispersal_units')[12][0]

In [ ]:
g=disperse(g=g, emission_model=emitter,transport_model=transporter, fungus_name='septoria',label='LeafElement')
g=infect(g=g,dt=1,infection_control_model=infection_controler,label='LeafElement')
g=update(g=g,dt=1,growth_control_model=growth_controler,label='LeafElement')


#### Simulation

In [ ]:
import pandas
import random as rd

from alinea.echap.architectural_reconstructions import EchapReconstructions
from alinea.echap.weather_data import get_weather

from alinea.alep.protocol import update, infect, disperse, update
from alinea.alep.septo3d_v2 import plugin_septoria
from alinea.alep.inoculation import RandomInoculation
from alinea.alep.infection_control import BiotrophDUProbaModel
from alinea.alep.growth_control import NoPriorityGrowthControl
from alinea.alep.dispersal_emission import SeptoriaRainEmission
from alinea.alep.dispersal_transport import SeptoriaRainDispersal
from alinea.alep.washing import RapillyWashing
from alinea.alep.architecture import set_properties, update_healthy_area, get_leaves
from alinea.alep.mini_models import leaf_wetness_rapilly
from alinea.alep.disease_outputs import plot_severity_by_leaf
from alinea.alep.disease_operation import generate_stock_du

from alinea.astk.plant_interface import grow_canopy
from alinea.astk.TimeControl import time_filter, rain_filter, time_control, IterWithDelays

%gui qt

In [ ]:
%pdb

In [ ]:
# wheat canopy initialization
echap = EchapReconstructions()
wheat = echap.get_reconstruction(name='Mercia', nplants=2)

g = wheat.setup_canopy(age=100)

# initialization of source of contamination
leaves = get_leaves(g)

source_leaf = g.node(leaves[0])

# initiatlisation of septoria model
septoria = plugin_septoria()
inoculator = RandomInoculation()
growth_controler = NoPriorityGrowthControl()
infection_controler = BiotrophDUProbaModel()
emitter = SeptoriaRainEmission(domain_area=wheat.domain_area)
transporter = SeptoriaRainDispersal()
washor = RapillyWashing()

# Read weather data, check data and add mistake data
weather = get_weather(variety='Mercia')
weather_index= pandas.date_range(start="2010-11-02",end="2011-05-09", freq='H')

weather.check(varnames=["temperature_air", "relative_humidity","rain","wind_speed","global_radiation","PPFD","vapor_pressure","degree_days","wetness"],models={'wetness': lambda data: leaf_wetness_rapilly(data.rain, data.relative_humidity, data.PPFD)})

# Define the schedule of calls for each model
every_h = time_filter(weather_index, delay=1)
every_24h = time_filter(weather_index, delay=24)
every_rain = rain_filter(weather_index, weather)
weather_timing = IterWithDelays(*time_control(weather_index, every_h, weather.data))
wheat_timing = IterWithDelays(*time_control(weather_index, every_24h, weather.data))
septo_timing = IterWithDelays(*time_control(weather_index, every_h, weather.data))
rain_timing = IterWithDelays(*time_control(weather_index, every_rain, weather.data))

In [ ]:
for i, controls in enumerate(zip(weather_timing, wheat_timing, septo_timing, rain_timing)):
    weather_eval, wheat_eval, septo_eval, rain_eval = controls

    # Get weather for date and add it as properties on leaves
    if weather_eval:
        set_properties(g,label = 'LeafElement',
                       temp = weather_eval.value.temperature_air[0],
                       wetness = weather_eval.value.wetness[0],
                       relative_humidity = weather_eval.value.relative_humidity[0],
                       wind_speed = weather_eval.value.wind_speed[0])
    if rain_eval:
        set_properties(g,label = 'LeafElement',
                       rain_intensity = rain_eval.value.rain.mean(),
                       rain_duration = len(rain_eval.value.rain) if rain_eval.value.rain.sum() > 0 else 0.)
    
    # Grow wheat canopy
    if wheat_eval:
        print('--', wheat_eval.value.index[-1], 'Growth canopy --')
        g,_ = grow_canopy(g, wheat, wheat_eval.value)
        positions = g.property('position_senescence')
        greens = g.property('is_green')
        areas = g.property('area')
        senesced_areas = g.property('senesced_area')
        leaves = get_leaves(g, label = 'LeafElement')
        vids = [leaf for leaf in leaves if leaf in g.property('geometry')]
        positions.update({vid:(0 if (positions[vid]==1 and not greens[vid]) or
                                    (positions[vid]>0 and round(areas[vid],5)==round(senesced_areas[vid],5))
                                    else positions[vid]) for vid in vids})
        # _=update(g= g,dt= wheat_eval.dt, growth_control_model= growth_controler)
                                    
    # Develop disease
    if septo_eval:
        print('--', septo_eval.value.index[-1], 'Disease development --')

        # Update g for the disease
        update_healthy_area(g, label = 'LeafElement')
        
        # Possibly refill pool of initial inoculum to simulate differed availability
        if rain_eval and i <= 700 and source_leaf.geometry!=None:
            dus = generate_stock_du(nb_dus=rd.randint(0,5), disease=septoria)
            try:
                source_leaf.dispersal_units += dus
            except:
                source_leaf.dispersal_units = dus
        
        # Update dispersal units and lesions
        infect(g, septo_eval.dt, infection_controler, label='LeafElement')
        #update(g, septo_eval.dt, growth_controler, sen_model, label='LeafElement')
        
    # Disperse and wash
    if rain_eval:
        print('--', rain_eval.value.index[-1], 'rain, disease dispertion --')
        if rain_eval.value.rain.mean()>0:
            g = disperse(g, emitter, transporter, "septoria", label='LeafElement')
           # wash(g, washor, rain_eval.value.rain.mean(), label='LeafElement')
            
    # Display outputs and save image
    if wheat_eval:
        scene = plot_severity_by_leaf(g, senescence=False, transparency=0.9)

## 3. Model Pesticide interception (Echap)

In [7]:
from matplotlib import cm

In [8]:
from alinea.echap.architectural_reconstructions import EchapReconstructions
from alinea.echap.interception_leaf import pesticide_applications, InterceptModel
from alinea.echap.interfaces import pesticide_interception
from alinea.echap.tests_nodes import plot_pesticide

from oawidgets.plantgl import PlantGL

### 3.1 initiate application data and interception model

In [9]:
application_data = """date,dose, product_name
2010-11-02 00:00:00, 0, bug
2010-11-02 1:00:00, 100, Opus
2010-11-02 2:00:00, 0, bug
2010-04-29 10:00:00, 100, Opus
2010-04-29 11:00:00, 0, bug
2010-05-22 10:00:00, 100, Opus
2010-05-22 11:00:00, 0, bug
"""
pest_calendar = pesticide_applications(application_data)
pest_calendar

,date,dose,product_name
date,,,
2010-04-29 10:00:00,2010-04-29 10:00:00,100,Opus
2010-04-29 11:00:00,2010-04-29 11:00:00,0,bug
2010-05-22 10:00:00,2010-05-22 10:00:00,100,Opus
2010-05-22 11:00:00,2010-05-22 11:00:00,0,bug
2010-11-02 00:00:00,2010-11-02 00:00:00,0,bug
2010-11-02 01:00:00,2010-11-02 1:00:00,100,Opus
2010-11-02 02:00:00,2010-11-02 2:00:00,0,bug


In [10]:
productsDB={'Opus': {'Epoxiconazole': 125}, 'Banko 500': {'Chlorothalonil': 500}} # name of application: molecule component and application dose
interception = InterceptModel(productsDB) # Model of pesitide appplication

### 3.2 Pesticide interception rmodel simulation

In [11]:
# canopy
echap = EchapReconstructions()
wheat=echap.get_reconstruction(name="Mercia",nplants=30)
g = wheat.setup_canopy(age=1500)

# pesticide interception
pesticide_interception(g=g, interception_model= interception, application_data=pest_calendar, label='LeafElement')
scene=plot_pesticide(g,colmap=cm.winter_r)
PlantGL(scene,group_by_color=False)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to 

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

{57: {'Epoxiconazole': 0.05783587699562419},
 58: {'Epoxiconazole': 0.807279091949029},
 59: {'Epoxiconazole': 1.0538566609179862},
 69: {'Epoxiconazole': 0.0},
 70: {'Epoxiconazole': 0.0},
 71: {'Epoxiconazole': 0.1719163582797668},
 81: {'Epoxiconazole': 0.0016207110688003742},
 82: {'Epoxiconazole': 0.001172328500627026},
 83: {'Epoxiconazole': 0.09352751293248475},
 93: {'Epoxiconazole': 0.0},
 94: {'Epoxiconazole': 0.2116081612749848},
 95: {'Epoxiconazole': 0.3671715254507451},
 105: {'Epoxiconazole': 0.05892172147557321},
 106: {'Epoxiconazole': 0.28121499261557875},
 107: {'Epoxiconazole': 0.3449541476882312},
 117: {'Epoxiconazole': 0.13989458746961858},
 118: {'Epoxiconazole': 0.22474378694463137},
 119: {'Epoxiconazole': 0.2853178362675505},
 157: {'Epoxiconazole': 0.014594715208522763},
 158: {'Epoxiconazole': 0.09665407347256899},
 159: {'Epoxiconazole': 0.04840079493898945},
 169: {'Epoxiconazole': 0.004874335057621091},
 170: {'Epoxiconazole': 0.11765977658495566},
 171:

### 3.3 Simulation canopy, growth and pesticide interception using weather data

In [12]:
import pandas
from matplotlib import cm

In [13]:
# canopy import
from alinea.echap.architectural_reconstructions import EchapReconstructions

# Weather import
from alinea.echap.weather_data import get_weather

from alinea.alep.mini_models import linear_degree_days
from alinea.alep.protocol import update
from alinea.alep.growth_control import NoPriorityGrowthControl

from alinea.astk.TimeControl import DegreeDayModel, thermal_time_filter, date_filter
from alinea.astk.TimeControl import IterWithDelays , time_control

# Pesticide import
from alinea.echap.interception_leaf import pesticide_applications, InterceptModel
from alinea.echap.interfaces import pesticide_interception
from alinea.echap.tests_nodes import plot_pesticide


In [14]:
# Weather parameter

## import weather data
weather = get_weather(variety="Mercia") # get weather data from echap for Mercia variety

##check weather data and add degree day variable using linear degree day model
weather.check(["degree_days",'temperature_air', 'PPFD', 'relative_humidity', 'wind_speed', 'rain', 'global_radiation', 'vapor_pressure'],models={'degree_days':linear_degree_days})

## weather index selection
weather_time_index = pandas.date_range(start ="2010-11-02", end='2011-05-09', freq='H')  # Range date index. among weather data select weather period for simulation

## canopy timing using degree day
TTmodel = DegreeDayModel(Tbase = 0) # Thermal times model
every_dd = thermal_time_filter(time_sequence=weather_time_index, weather=weather, model=TTmodel, delay = 15) # allows to filter thermal time data here each 15 degree day
canopy_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_dd, data=weather.data)) # create data from weatherdata according to filter

# Pesticide model

# Pesticide data and interception model
application_data = """date,dose, product_name
2010-11-02 00:00:00, 0, bug
2010-11-02 1:00:00, 100, Opus
2010-11-02 2:00:00, 0, bug
2010-04-29 10:00:00, 100, Opus
2010-04-29 11:00:00, 0, bug
2010-05-22 10:00:00, 100, Opus
2010-05-22 11:00:00, 0, bug
"""
productsDB={'Opus': {'Epoxiconazole': 125}, 'Banko 500': {'Chlorothalonil': 500}} # name of application: molecule component and application dose
interception = InterceptModel(productsDB) # Model of pesitide appplication

#pestide timing
pest_calendar = pesticide_applications(application_data)
every_pest = date_filter(time_sequence=weather_time_index, time_data=pest_calendar)
pest_timing = IterWithDelays(*time_control(time_sequence=weather_time_index, eval_filter=every_pest, data=pest_calendar))

# growth control
growth_controler = NoPriorityGrowthControl()

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1124: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  return self._get_label(key, axis=axis)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)


In [15]:
echap = EchapReconstructions() # load architectural parameter reconstruction with echap data
wheat = echap.get_reconstruction(name="Mercia", nplants=2,seed=0)
g = wheat.setup_canopy(age=500)

for i,controls in enumerate(zip(canopy_timing, pest_timing)):
    canopy_iter, pest_iter = controls
    
    if canopy_iter.eval:
        print('--', canopy_iter.value.index[-1], '--')
        print('-- update microclimate / canopy --')
        g = wheat.grow(g, canopy_iter.value)
        _=update(g, canopy_iter.dt, growth_controler)
        _=wheat.plot(g)

    if pest_iter.eval:
        print('--', pest_iter.value.index[-1], '--')
        print('-- update microclimate / pesticide --')
        
        _=pesticide_interception(g, interception, pest_iter.value, label='LeafElement')
        _=plot_pesticide(g)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to 

-- 2010-11-03 09:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-02 00:00:00 --
-- update microclimate / pesticide --
-- 2010-11-02 01:00:00 --
-- update microclimate / pesticide --
-- 2010-11-02 02:00:00 --
-- update microclimate / pesticide --
-- 2010-11-04 10:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-05 12:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-06 20:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-09 03:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-11 13:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-12 17:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-13 18:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-14 23:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-17 13:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-20 01:00:00+00:00 --
-- update microclimate / canopy --
-- 2010-11-22 17:00:00+00:00 --
-- update microclimate / canopy --
-- 2

## 4. Pesticide penetration model (echap)

In [ ]:
from alinea.echap.milne_leaf import PenetratedDecayModel
from alinea.echap.interfaces import pesticide_penetrated_decay

from alinea.astk.TimeControl import time_filter , IterWithDelays


### 3.1 initiate penetration model

In [ ]:
# weather data
## import weather data
weather = get_weather(variety="Mercia") # get weather data from echap for Mercia variety

##check weather data and add degree day variable using linear degree day model
weather.check(["degree_days",'temperature_air', 'PPFD', 'relative_humidity', 'wind_speed', 'rain', 'global_radiation', 'vapor_pressure'],models={'degree_days':linear_degree_days})

## weather index selection
weather_time_index = pandas.date_range(start ="2010-11-02", end='2011-05-09', freq='H')  # Range date index. among weather data select weather period for simulation

## Weather data hours
every_hours = time_filter(time_sequence=weather_time_index, delay = 1)
pest_penetration_dose_timing = IterWithDelays(*time_control(weather_time_index, every_hours, weather.data))

decay_model = PenetratedDecayModel() 
g = pesticide_penetrated_decay(g=g, decay_model= decay_model,weather_data=weather.data,label="LeafElement")


In [ ]:
g.property('penetrated_doses')

## 5. pesticide washing model